<a href="https://colab.research.google.com/github/thesis17/Afaan-Oromoo-chatGPT/blob/main/Afaan_Oromoo_chatGPT_of_Fine_Tuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

file = json.load(open('/content/drive/MyDrive/Complete_Thesis/afaan_oromoo_chatgpt_format_dataset.json', "r"))
print(file[1])

{'idx': 1, 'dhukkuba': 'Poolippii funyoo sagalee', 'Mallattoo': "['Sagalee hoarse', 'Dhukkubbii qoonqoo', 'Dubbuu dubbachuuf rakkachuu', 'Qufaa', 'Dhiphina hidhii', 'Dhiphina qoonqoo', 'Dhaga'uu hir'achuu', 'Qonqoo keessatti citaa', 'Qonoon dhiphina itti dhaga'ama', 'Rakkoon liqimsuu', 'Dhiita'uu gogaa', 'Fincaan qabachuu']", 'sababa': 'beclomethasone oomisha hidhii,', 'QormaataFiHojimaata': "['Tiraakiyooskoopii fi laariingoskoopii baayoopsii waliin', 'Madaallii wal'aansa hojii (Yaala dubbii)', 'Hojimaata adda baasuu biroo (af-gaaffii; madaallii; marii)', 'Shaakala wal'aansa qaamaa (Shaakala)', 'Hojimaata adda baasuu hidhii irratti; afaanii fi pharynx', 'Yaala qaamaa fi deebisanii dhaabuu biroo', 'Qorannoo fi wal'aansa ijaa fi otologic']", 'Qorichoota Beekamoo': "['Esomeprazole (Nexium)', 'Oomisha Hidhii Beclomethasone', 'Nicotine', 'Azelastine Nasal', 'Phenylephrine (Duramax)', 'Rabeprazole (Aciphex)', 'Vinorelbine (Navelbine)', 'Vitamin A', ' Adalimumab (Humira)', 'Rituximab'] jedhu 

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.8.10: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import Dataset
import json

def format_prompt(example):
    input_text = f"Dhukkuba: {example['dhukkuba']}\nSababa: {example['sababa']}"
    output_data = {
        "Mallattoo": example['Mallattoo'],
        "QormaataFiHojimaata": example['QormaataFiHojimaata'],
        "Qorichoota Beekamoo": example['Qorichoota Beekamoo']
    }
    return f"### Input: {input_text}\n### Output: {json.dumps(output_data, ensure_ascii=False)}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,  # Effective batch size = 16
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/796 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 796 | Num Epochs = 3 | Total steps = 150
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,entropy
25,2.699300,0
50,1.551100,No Log
75,1.208900,No Log
100,1.135000,No Log
125,0.983400,No Log
150,0.943900,No Log


In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
example_data = {'idx': 1, 'dhukkuba': 'Poolippii funyoo sagalee', 'Mallattoo': "['Sagalee hoarse', 'Dhukkubbii qoonqoo', 'Dubbuu dubbachuuf rakkachuu', 'Qufaa', 'Dhiphina hidhii', 'Dhiphina qoonqoo', 'Dhaga\'uu hir\'achuu', 'Qonqoo keessatti citaa', 'Qonoon dhiphina itti dhaga\'ama', 'Rakkoon liqimsuu', 'Dhiita\'uu gogaa', 'Fincaan qabachuu']", 'sababa': 'beclomethasone oomisha hidhii,', 'QormaataFiHojimaata': "['Tiraakiyooskoopii fi laariingoskoopii baayoopsii waliin', 'Madaallii wal\'aansa hojii (Yaala dubbii)', 'Hojimaata adda baasuu biroo (af-gaaffii; madaallii; marii)', 'Shaakala wal\'aansa qaamaa (Shaakala)', 'Hojimaata adda baasuu hidhii irratti; afaanii fi pharynx', 'Yaala qaamaa fi deebisanii dhaabuu biroo', 'Qorannoo fi wal\'aansa ijaa fi otologic']", 'Qorichoota Beekamoo': "['Esomeprazole (Nexium)', 'Oomisha Hidhii Beclomethasone', 'Nicotine', 'Azelastine Nasal', 'Phenylephrine (Duramax)', 'Rabeprazole (Aciphex)', 'Vinorelbine (Navelbine)', 'Vitamin A', ' Adalimumab (Humira)', 'Rituximab'] jedhu kan qabu ta'uu ibsameera}"}

user_input_string = f"### Input: Dhukkuba: {example_data['dhukkuba']}\nSababa: {example_data['sababa']}\n### Output:"

messages = [
    {"role": "user", "content": user_input_string}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> ### Input: Dhukkuba: Poolippii funyoo sagalee
Sababa: beclomethasone oomisha hidhii,
### Output:<|end|><|assistant|> {"Mallattoo": "['Dhukkubbiin garaa qara', 'Dhukkubbiin garaa gubuu', 'Dhukkubbiin garaa gubbaa', 'Dhukkubbiin garaa gubbaa', 'Dhukkubbii hunda', 'Dhukkubbii garaa qara', 'Dhukkubbii harkaa', 'Dhukkubbii dugdaa', 'Dhukkubbii cinaa', 'Dhukkubbii garaa gadii', 'Dhukkubbii dugdaa gadii', 'Dhukkubbiin garaa gubbaa']", "QormaataFiHojimaata": "['Qorannoo gogaa qaamaa guutuu raawwatame (ML)', 'Xiinxala fincaanii', 'Hojimaata adda baasuu biroo (af-gaaffii; madaallii; marii)


In [ ]:
model.save_pretrained_gguf("gguf_model_afaan_oromoo_chatGPT", tokenizer, quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.3G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.7 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:01<00:00, 17.08it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving gguf_model_afaan_oromoo_chatGPT/pytorch_model-00001-of-00002.bin...
Unsloth: Saving gguf_model_afaan_oromoo_chatGPT/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at gguf_model_afaan_oromoo_chatGPT into f16 GGUF format.
The output location will be /content/gguf_model_afaan_oromoo_chatGPT/unsloth.F16.gguf
This might take 3 minutes...


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
!pip uninstall protobuf -y
!pip install protobuf==3.20.3

Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model_afaan_oromoo_chatGPT") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model_afaan_oromoo_chatGPT", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'